In [1]:
import duckdb

In [8]:
DB_PATH = 'duckdb.db'

In [9]:
def analyze_duckdb_database():

    # Connect to the database
    conn = duckdb.connect(DB_PATH)

    # Get list of all schemas
    schemas = conn.execute("""
                SELECT schema_name 
                FROM information_schema.schemata 
                WHERE schema_name NOT IN ('information_schema', 'pg_catalog')
                ORDER BY schema_name
            """).fetchall()

    for schema in schemas:
        schema_name = schema[0]

        #if schema_name == 'main':
        #    continue

        print(f"Schema: {schema_name}")
        print("-" * 40)
        
        # Get list of tables in the schema
        tables = conn.execute(f"""
            SELECT table_name 
            FROM information_schema.tables 
            WHERE table_schema = '{schema_name}'
            AND table_type = 'BASE TABLE'
            ORDER BY table_name
        """).fetchall()

        # For each table, get records count
        for table in tables:
            table_name = table[0]
            
            # Get the number of rows in the table
            row_count = conn.execute(f"""
                SELECT COUNT(*) FROM "{schema_name}"."{table_name}"
            """).fetchone()[0]

            print(f"  Table: {table_name}: row count {row_count:,}")

        print("-" * 40)

    conn.close()

In [10]:
analyze_duckdb_database()

Schema: ingestion
----------------------------------------
  Table: _dlt_loads: row count 2
  Table: _dlt_pipeline_state: row count 6
  Table: _dlt_version: row count 3
  Table: payment_lookup: row count 7
  Table: trips: row count 1,287,699
----------------------------------------
Schema: main
----------------------------------------
----------------------------------------
Schema: main
----------------------------------------
----------------------------------------
Schema: main
----------------------------------------
----------------------------------------
